In [1]:
import torch
import intel_extension_for_pytorch as ipex
import torchvision
from torch import nn
import sys
import os

/home/taosy/repo/pytorch/vision/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: ''If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/taosy/bin/miniconda3/envs/pt-39-6/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(torch.__version__)
print(torchvision.__version__)

m=torchvision.models.resnet50()
print(os.getpid())


2.1.0a0+git6d20b39
0.16.0a0+cab01fc


1176473


In [32]:
m=torchvision.models.resnet50()
m=m.eval()
inputs=torch.randn(5, 3, 224, 224)
m1=torch.jit.trace(m,inputs)
m1=torch.jit.freeze(m1)
m2=torch.fx.symbolic_trace(m)
m3=ipex.optimize(m)
with torch.no_grad():
    m4=torch.jit.trace(m3.eval(),inputs)
    m4=torch.jit.freeze(m4)

print(type(m1))
print(type(m2))
print(type(m3))
print(type(m4))

<class 'torch.jit._script.RecursiveScriptModule'>
<class 'torch.fx.graph_module.GraphModule.__new__.<locals>.GraphModuleImpl'>
<class 'torch.fx.graph_module.GraphModule.__new__.<locals>.GraphModuleImpl'>
<class 'torch.jit._script.RecursiveScriptModule'>


In [ ]:
print(m1.graph)

In [42]:
import torch
import intel_extension_for_pytorch as ipex

class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
    def forward(self, a):
        b = torch.conv2d(a, torch.randn(1, 1, 1, 1)) # not fusible
        x = torch.mul(b, b)                          # fusible
        y = torch.sin(x)                             # fusible
        z = torch.mul(y, y)                          # fusible
        return z

m=Model().eval()

inputs = torch.randn(1, 1, 128, 128)
m1=torch.jit.trace(m,inputs)
m1=torch.jit.freeze(m1)
m2=torch.fx.symbolic_trace(m)
m3=ipex.optimize(m)
m4=torch.jit.trace(m3.eval(),inputs)
m4=torch.jit.freeze(m4)

print(type(m1))
print(type(m2))
print(type(m3))
print(type(m4))
print(m1.graph)
print(m4.graph)

<class 'torch.jit._script.RecursiveScriptModule'>
<class 'torch.fx.graph_module.GraphModule.__new__.<locals>.GraphModuleImpl'>
<class 'torch.fx.graph_module.GraphModule.__new__.<locals>.GraphModuleImpl'>
<class 'torch.jit._script.RecursiveScriptModule'>
graph(%self : __torch__.___torch_mangle_7623.Model,
      %a : Tensor):
  %29 : bool = prim::Constant[value=1]() # /tmp/ipykernel_1176473/1623011307.py:8:0
  %37 : int[] = prim::Constant[value=[0, 0]]()
  %36 : int[] = prim::Constant[value=[1, 1]]()
  %10 : bool = prim::Constant[value=0]() # /tmp/ipykernel_1176473/1623011307.py:8:0
  %9 : Device = prim::Constant[value="cpu"]() # /tmp/ipykernel_1176473/1623011307.py:8:0
  %7 : NoneType = prim::Constant()
  %35 : int[] = prim::Constant[value=[1, 1, 1, 1]]()
  %2 : int = prim::Constant[value=1]() # /tmp/ipykernel_1176473/1623011307.py:8:0
  %11 : Tensor = aten::randn(%35, %7, %7, %9, %10) # /tmp/ipykernel_1176473/1623011307.py:8:0
  %b : Tensor = aten::_convolution(%a, %11, %7, %36, %37, %

/home/taosy/repo/pytorch/pytorch-nightly-pt39-6/torch/jit/_trace.py:1093: TracerWarning: Trace had nondeterministic nodes. Did you forget call .eval() on your model? Nodes:
	%13 : Float(1, 1, 1, 1, strides=[1, 1, 1, 1], requires_grad=0, device=cpu) = aten::randn(%8, %9, %10, %11, %12) # /tmp/ipykernel_1176473/1623011307.py:8:0
This may cause errors in trace checking. To disable trace checking, pass check_trace=False to torch.jit.trace()
  _check_trace(
/home/taosy/repo/pytorch/pytorch-nightly-pt39-6/torch/jit/_trace.py:1093: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 15339 / 16384 (93.6%)
Greatest absolute difference: 0.9985638820799068 at index (0, 0, 74, 80) (up to 1e-05 allowed)
Greatest relative difference: 1715.2421240867543 at index (0, 0, 36, 7) (up to 1e-05 allowed)
  _check_trace(


In [37]:
print(type(m1))
print(m.graph)


<class 'torch.jit.ScriptFunction'>


AttributeError: 'function' object has no attribute 'graph'

In [21]:
m.print_readable()

class M(torch.nn.Module):
    def forward(self, x):
        # No stacktrace found for following nodes
        relu_1 = torch.nn.functional.relu(x, inplace = False);  x = None
        add = relu_1 + 1.0;  relu_1 = None
        return add
        


'class M(torch.nn.Module):\n    def forward(self, x):\n        # No stacktrace found for following nodes\n        relu_1 = torch.nn.functional.relu(x, inplace = False);  x = None\n        add = relu_1 + 1.0;  relu_1 = None\n        return add\n        '

In [19]:
print(type(m))
print(m.graph)

<class 'torch.fx.graph_module.GraphModule.__new__.<locals>.GraphModuleImpl'>
graph():
    %x : [num_users=1] = placeholder[target=x]
    %relu_1 : [num_users=1] = call_function[target=torch.nn.functional.relu](args = (%x,), kwargs = {inplace: False})
    %add : [num_users=1] = call_function[target=operator.add](args = (%relu_1, 1.0), kwargs = {})
    return add
